# DenseNet201 기반 이미지 분류 (10-Fold CV + 증강)

In [ ]:
# 사용 라이브러리 버전
# numpy : 2.2.4
# pandas : 2.2.3
# torch : 2.6.8+cu124
# torchvision : 0.21.0+cu124
# timm : 1.0.15
# scikit-learn : 1.6.1
# tqdm : 4.67.1

In [ ]:
# 경고 무시 설정 및 필수 라이브러리 임포트
import warnings
warnings.filterwarnings('ignore')  # 경고 메시지를 무시하여 출력 깔끔하게 유지

import numpy as np                  # 수치 연산을 위한 NumPy
import pandas as pd                 # 데이터 처리 및 CSV 입출력용 pandas

import torch                        # PyTorch 메인 패키지
import torch.nn as nn               # 신경망 레이어 및 손실 함수
import torch.optim as optim         # 최적화 알고리즘

from torch.utils.data import Dataset, DataLoader, Subset  # 데이터셋, 로더 유틸리티

from torchvision.transforms import Compose, ToPILImage, Resize, ToTensor, Normalize, RandomHorizontalFlip, RandomVerticalFlip
# 이미지 전처리 및 증강을 위한 torchvision.transforms

import timm                         # PyTorch 모델 라이브러리

from sklearn.preprocessing import LabelEncoder            # 레이블 인코딩
from sklearn.model_selection import StratifiedKFold       # 층화 K-Fold 교차 검증

from tqdm import tqdm              # 진행 상황 표시용 tqdm


## 1. 하이퍼파라미터 설정

In [ ]:
# Baseline 하이퍼파라미터 정의
N_EPOCHS = 100     # 에폭 수
BATCH_SIZE = 8     # 배치 크기
LR = 5e-5          # 학습률
N_FOLDS = 10       # 교차 검증 폴드 수
SEED = 42          # 랜덤 시드 고정


## 2. 데이터 로드 및 라벨 전처리

In [ ]:
# CSV 파일에서 학습/테스트 데이터 로드
train = pd.read_csv("/data/train.csv")  # 학습 데이터
test = pd.read_csv("/data/test.csv")    # 테스트 데이터

# 문자열 레이블을 숫자로 변환
encoder = LabelEncoder()
train['label'] = encoder.fit_transform(train['label'])


## 3. CustomDataset 클래스 정의

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, pixel_df, label_df=None, transform=None):
        # DataFrame을 인덱스 초기화하여 깔끔하게 관리
        self.pixel_df = pixel_df.reset_index(drop=True)
        self.label_df = label_df.reset_index(drop=True) if label_df is not None else None
        self.transform = transform

    def __len__(self):
        # 데이터셋 크기를 반환
        return len(self.pixel_df)
    
    def __getitem__(self, idx):
        # 픽셀 데이터를 32x32 이미지 배열로 변환
        image = self.pixel_df.iloc[idx].values.astype(np.uint8).reshape(32, 32)
        image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # (1, 32, 32) 형태로 변환
        if self.transform:
            image = self.transform(image)  # Transform(증강/전처리) 적용
        if self.label_df is not None:
            label = torch.tensor(self.label_df.iloc[idx], dtype=torch.long)
            return image, label
        else:
            # 테스트 시에는 레이블이 없으므로 이미지만 반환
            return image


## 4. 데이터 증강 및 전처리 정의

In [ ]:
# 학습용 증강: 좌우/상하 뒤집기 + 크기 조정 + 정규화
train_transform = Compose([
    ToPILImage(),                         # 텐서를 PIL 이미지로 변환
    Resize((224, 224)),                   # 모델 입력 크기로 Resize
    RandomHorizontalFlip(p=0.5),          # 랜덤 좌우 뒤집기
    RandomVerticalFlip(p=0.5),            # 랜덤 상하 뒤집기
    ToTensor(),                           # 다시 텐서로 변환
    Normalize(mean=[0.5], std=[0.5]),     # 정규화
])

# 검증/테스트용 전처리: 증강 없이 Resize + 정규화
val_transform = Compose([
    ToPILImage(),
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=[0.5], std=[0.5]),
])


## 5. Device 설정 및 DataLoader 파라미터

In [ ]:
# GPU 사용 여부 판단
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# DataLoader 공통 파라미터
loader_params = {
    'batch_size': BATCH_SIZE,
    'num_workers': 0,   # 환경에 따라 worker 수 조정
    'pin_memory': True  # GPU 메모리 활용 최적화
}


## 6. 10-Fold 교차 검증 학습 루프

In [ ]:
# StratifiedKFold로 폴드 분할 객체 생성
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

all_fold_preds = []  # 폴드별 테스트 예측 저장
fold_idx = 1

for train_idx, valid_idx in skf.split(train.iloc[:, 2:], train['label']):
    print(f"--- Fold {fold_idx} 시작 ---")
    
    # 학습/검증 데이터셋 분리
    train_dataset = CustomDataset(train.iloc[train_idx, 2:], train.iloc[train_idx, 1], transform=train_transform)
    valid_dataset = CustomDataset(train.iloc[valid_idx, 2:], train.iloc[valid_idx, 1], transform=val_transform)
    
    train_loader = DataLoader(train_dataset, shuffle=True, **loader_params)
    valid_loader = DataLoader(valid_dataset, shuffle=False, **loader_params)
    
    # 모델 초기화: DenseNet201, 입력 채널 1, 클래스 수 10
    model = timm.create_model("densenet201", pretrained=False, num_classes=10, in_chans=1).to(device)
    
    # 손실 함수 및 최적화 함수
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=LR)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=N_EPOCHS)
    
    best_loss = float('inf')
    best_state = None
    
    # 에폭 반복
    for epoch in range(N_EPOCHS):
        print(f"Fold {fold_idx} Epoch {epoch+1}/{N_EPOCHS}")
        
        # ----- 학습 단계 -----
        model.train()
        running_loss = 0.0
        for images, labels in tqdm(train_loader, desc="Training", leave=False):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
        train_loss = running_loss / len(train_loader.dataset)
        
        # ----- 검증 단계 -----
        model.eval()
        running_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for images, labels in tqdm(valid_loader, desc="Validation", leave=False):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                running_loss += loss.item() * images.size(0)
                _, preds = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (preds == labels).sum().item()
        val_loss = running_loss / len(valid_loader.dataset)
        val_acc = correct / total
        
        print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc*100:.2f}%")
        
        # 최적 모델 상태 저장
        if val_loss < best_loss:
            best_loss = val_loss
            best_state = model.state_dict()
        
        scheduler.step()
    
    # 최적 가중치 로드
    model.load_state_dict(best_state)
    print(f"Fold {fold_idx} 완료. 최적 모델 로드.")
    
    # 테스트 데이터 예측
    test_loader = DataLoader(CustomDataset(test.iloc[:, 1:], transform=val_transform), shuffle=False, **loader_params)
    fold_preds = []
    model.eval()
    with torch.no_grad():
        for images in tqdm(test_loader, desc="Inference", leave=False):
            images = images.to(device)
            outputs = model(images)
            fold_preds.append(outputs.cpu().numpy())
    all_fold_preds.append(np.concatenate(fold_preds, axis=0))
    
    fold_idx += 1


## 7. 예측 평균 및 제출 파일 생성

In [ ]:
# 폴드별 예측 결과 평균
avg_test_preds = np.mean(np.array(all_fold_preds), axis=0)
pred_indices = np.argmax(avg_test_preds, axis=1)

# 숫자 레이블을 원본 문자열 레이블로 변환
pred_labels = encoder.inverse_transform(pred_indices)

# 제출 파일 생성
submission = pd.read_csv("/data/sample_submission.csv")
submission['label'] = pred_labels
submission.to_csv("/data/submission_dense201_cv_aug.csv", index=False, encoding="utf-8-sig")

print("File saved")
